
对GATK DNAseq 分析流程进行总结，使用的gatk版本为GAT4.1.8.0

数据下载：

+ GATK介绍

https://gatk.broadinstitute.org/hc/en-us/articles/360036194592-Getting-started-with-GATK4

+ 下载链接

https://github.com/broadinstitute/gatk/releases

4.1.8： https://github.com/broadinstitute/gatk/releases/download/4.1.8.0/gatk-4.1.8.0.zip

+ bwa下载链接

https://sourceforge.net/projects/bio-bwa/files/bwa-0.7.17.tar.bz2/download

https://sourceforge.net/projects/bio-bwa/files/

# Germline SNV分析

一般是多个样本进行germline分析，这样会更准确；

也有单个样本的germline分析，需要使用机器学习算法；

<img src="./figures/fig1.png">

分析流程主要包括：

+ 数据预处理: BWA

+ 标记重复: MarkDuplicates

+ 碱基重新矫正：BaseRecalibrator/ApplyBQSR

+ variantCalling: haplotyperCaller，由bam产生vcf/g.vcf文件

+ JointCall: CombineGVCFs,将多个样本的g.vcf结合，产生vcf文件

+ Genotyper： GenotypeGVCFs对上一步产生的vcf进行genotyping

+ 使用VQSR进行矫正：VariantRecalibrator，ApplyVQSR

+ 或者使用hard-filter进行过滤：variantFiltration

参考文章：https://gatk.broadinstitute.org/hc/en-us/articles/360035535932-Germline-short-variant-discovery-SNPs-Indels-

## 数据预处理

关于数据预处理，GATK官方也给出了The Best Practices:

https://gatk.broadinstitute.org/hc/en-us/articles/360035535912

+ bwa

+ MarkDuplicates 

+ SortSam

## Variantion callings


```
$gatk HaplotypeCaller \
-R ~/gatk_files/human_g1k_v37.fasta \
-L ~/gatk_files/giab_agilent_sureselect_v5.list.bed \
-D ~/gatk_files/dbsnp_138.b37.vcf \
-I 151002_7001448_0359_AC7F6GANXX_Sample_HG003-EEogPU_v02-KIT-Av5_TCTTCACA_L008.posiSrt.markDup.bam  \
-O interval4180/hg003.g.vcf.gz \
-ERC GVCF 
```

## JointCall

```
$gatk CombineGVCFs \
-R ~/gatk_files/human_g1k_v37.fasta \
--variant hg002.g.vcf.gz \
--variant hg003.g.vcf.gz \
--variant hg004.g.vcf.gz \
-O corhort.g.vcf.gz
```

## Genotyper

```
$gatk GenotypeGVCFs \
-R ~/gatk_files/human_g1k_v37.fasta \
-V corhort.g.vcf.gz \
-O hg020304.vcf.gz
```

## VQSR 

### 可选步骤
把SNP和indel分别取出来进行VQSR分析（也可以在一个文件中，但是要分两步做）

```
$gatk SelectVariants \
-R ~/gatk_files/human_g1k_v37.fasta \
-V hg020304.vcf.gz \
--select-type-to-include SNP \
-O hg020304_SNP.vcf.gz

$gatk SelectVariants \
-R ~/gatk_files/human_g1k_v37.fasta \
-V hg020304.vcf.gz \
--select-type-to-include INDEL \
-O hg020304_INDEL.vcf.gz
```


### VQSR-snp

```
hapmap=~/gatk_files/hapmap_3.3.b37.vcf
 omni=~/gatk_files/1000G_omni2.5.b37.vcf
 kG=~/gatk_files/1000G_phase1.snps.high_confidence.b37.vcf
 dbsnp=~/gatk_files/dbsnp_138.b37.vcf

#SNP
$gatk VariantRecalibrator \
   -R  ~/gatk_files/human_g1k_v37.fasta \
   -V hg020304_SNP.vcf.gz \
   --resource:hapmap,known=false,training=true,truth=true,prior=15.0 $hapmap  \
   --resource:omni,known=false,training=true,truth=false,prior=12.0 $omni \
   --resource:1000G,known=false,training=true,truth=false,prior=10.0 $kG \
   --resource:dbsnp,known=true,training=false,truth=false,prior=2.0 $dbsnp \
   -an QD -an MQ -an MQRankSum -an ReadPosRankSum -an FS -an SOR \
   -mode SNP \
   -O output_SNP_vcf.recal \
   --tranches-file output_SNP.tranches
```


### ApplyVQSR-snp

```
$gatk ApplyVQSR -R ~/gatk_files/human_g1k_v37.fasta \
   -V hg020304_SNP.vcf.gz \
   -O ouput_SNP_recal.vcf.gz \
   --truth-sensitivity-filter-level 99.0 \
   --tranches-file output_SNP.tranches \
   --recal-file output_SNP_vcf.recal \
   -mode SNP
```

### VQSR-Indel

```
kgindel=~/gatk_files/1000G_phase1.indels.b37.vcf
 stdinde=~/gatk_files/Mills_and_1000G_gold_standard.indels.b37.vcf
 $gatk VariantRecalibrator \
   -R  ~/gatk_files/human_g1k_v37.fasta \
   -V ouput_INDEL_vcf.gz \
   --resource:mills,known=false,training=true,truth=true,prior=15.0 $stdinde  \
   -an QD -an MQ -an MQRankSum -an ReadPosRankSum -an FS -an SOR \
   -mode INDEL \
   -O output_INDEL_vcf.recal \
   --tranches-file output_INDEL.tranches
```


### ApplyVQSR-indel

```
$gatk ApplyVQSR -R ~/gatk_files/human_g1k_v37.fasta \
   -V hg020304_SNP.vcf.gz \
   -O ouput_indel_recal.vcf.gz \
   --truth-sensitivity-filter-level 99.0 \
   --tranches-file output_INDEL.tranches \
   --recal-file output_INDEL_vcf.recal \
   -mode indel
```

### 可选步骤 
将分开的snp indel进行合并
```
$gatk MergeVcfs \
-I ouput_SNP_recal.vcf.gz \
-I output_INDEL_hard_filtered.vcf.gz \
-O hg020304_SNP_INDEL_filtered.vcf.gz
```

## hard-filter

采用固定参数，对indel的结果进行硬过滤，可在--filter-name中添加字段

```
$gatk VariantFiltration \
-R ~/gatk_files/human_g1k_v37.fasta \
-V hg020304_INDEL.vcf.gz \
-O output_INDEL_hard_filtered.vcf.gz \
--filter-name "QD2.0" --filter-expression "QD < 2.0" \
--filter-name "FS200.0" --filter-expression "FS > 200.0" \
--filter-name "ReadPosRankSum" --filter-expression "ReadPosRankSum < -20.0" 
```

# Single sample Germline SNV分析

参考文档：
https://gatk.broadinstitute.org/hc/en-us/articles/360035535932-Germline-short-variant-discovery-SNPs-Indels-

<img src="./figures/fig2.png">

# Somtaic SNV分析

需要normal文件，也可以tumor-only分析

<img src="./figures/fig3.png">

分析流程主要包括：

+ 数据预处理: BWA

+ 标记重复: MarkDuplicates

+ 碱基重新矫正：BaseRecalibrator/ApplyBQSR

+ Mutect2，与germline不同，这里使用Mutect2进行variant calling

+ calculate contamination: 给出一个cross-sample contamination，一些潜在的非somatic 突变

+ variants 过滤：FilterMutectCalls对产生的vcf文件进行过滤

+ 突变注释：Funcotator，对突变位点加上注释信息

参考文档：https://gatk.broadinstitute.org/hc/en-us/articles/360035894731-Somatic-short-variant-discovery-SNVs-Indels-

## 数据预处理

关于数据预处理，GATK官方也给出了The Best Practices:

https://gatk.broadinstitute.org/hc/en-us/articles/360035535912

+ bwa

+ MarkDuplicates 

+ SortSam

## Vairantion callings(Mutect2)

+ tumor with matched normal

参考文档： https://gatk.broadinstitute.org/hc/en-us/articles/360046788432-Mutect2

```
     gatk Mutect2 \
     -R reference.fa \
     -I tumor.bam \
     -I normal.bam \
     -normal normal_sample_name \
     --germline-resource af-only-gnomad.vcf.gz \
     --panel-of-normals pon.vcf.gz \
     -O somatic.vcf.gz
```

+ tumor-only 

```
gatk Mutect2 \
   -R reference.fa \
   -I sample.bam \
   -O single_sample.vcf.gz
```

+ tumor-only with PoN(panel of normals)
```
  gatk Mutect2 \
  -R reference.fa \
  -I sample.bam \
  --germline-resource af-only-gnomad.vcf.gz \
  --panel-of-normals pon.vcf.gz \
  -O single_sample.vcf.gz
```
<p style="font-size:10px;color:blue;"><b>af-only-gnomad.vcf.gz</b> 是funcotator中使用到的一个data source</p>

+ Mitochondrial mode
```
gatk Mutect2 \
  -R reference.fa \
  -L chrM \
  --mitochondria-mode \
  -I mitochondria.bam \
  -O mitochondria.vcf.gz
```

+ Force-calling mode
```
gatk Mutect2 \
   -R reference.fa \
   -I sample.bam \
   -alleles force-call-alleles.vcf
   -O single_sample.vcf.gz
```

## Estimate contamination reads

包含两步

### GetPileupSummaries
需要两个输入文件
+ germline variant sites VCF,比如gnomAD resource
+ bam文件
```
gatk GetPileupSummaries \
   -I normal.bam \
   -V gnomad.vcf.gz \
   -L common_snps.interval_list \
   -O pileups.table
```

+ 产生的结果是这个样子
```
contig	position	ref_count	alt_count	other_alt_count	allele_frequency
 chr6	29942512	9	0	0	0.063
 chr6	29942517	13	1	0	0.062
 chr6	29942525	13	7	0	0.063
 chr6	29942547	36	0	0	0.077
```


### CalculateContamination

计算the fraction of reads coming from cross-sample contamination

+ tumor-only 
```
 gatk CalculateContamination \
   -I pileups.table \
   -O contamination.table
```

+ matched normal 

```
gatk CalculateContamination \
   -I tumor-pileups.table \
   -matched normal-pileups.table \
   -O contamination.table

```

## Filter mutations

对mutect2得到的vcf结果中的突变sites进行过滤

```
 gatk FilterMutectCalls \
   -R reference.fasta \
   -V somatic.vcf.gz \
   --contamination-table contamination.table \
   --tumor-segmentation segments.tsv \
   -O filtered.vcf.gz
```
<p style="font-size:10px;color:blue;"><b>segments.tsv, contamination.table</b> 均可由CalculateContamination计算得到</p>

## 突变注释

采用Funcotator对获得的突变位点进行注释

参考文档：https://gatk.broadinstitute.org/hc/en-us/articles/360046786432-Funcotator

使用方法看起来很简单，但是还需要进行学习，比如和annovar的区别，注释结果的结果。这个注释工具有个很大的resource data，30G~40G，需要下载解压使用。

+ germline/somatic

+ hg19/hg38

```
   ./gatk Funcotator \
   -R reference.fasta \
   -V input.vcf \
   -O outputFile \
   --output-file-format MAF \
   --data-sources-path dataSourcesFolder/ \
   --ref-version hg19
```